In [1]:
import pandas as pd
import os

In [2]:
# Not all listed here, only some Q files
list = ['07CE010','07CE013','07DA007','07DA023','07DA024','07DA025','07DA026','07DA027','07DA028','07DA029','07DA030',
        '07DA032','07DA033','07DA034','07DA035','07DA037','07DA038','07DA039','07DA040','07DA041','07DA042','07DA044',
        '07DA045','07DB002','07DB003','07DB006','07DC003','07DC004','07DD001',
        '07CD008','07CD009','07CE005','07CE007','07CE008']

In [3]:
cols = ['Station', 'Date', 'Discharge', 'Discharge Flag']
cols2 = [' ID','PARAM','Date','Flow(m³/s)','SYM']

In [4]:
df = pd.read_csv(os.getcwd() + os.sep + 'osm-annual-report-master' + os.sep + 'RAMP_Q.csv')

In [5]:
df = df[cols]

In [6]:
df = df.rename(columns={'Station': ' ID', 'Discharge': 'Flow(m³/s)', 'Discharge Flag': 'SYM'})

In [7]:
df['PARAM'] = 1

In [8]:
df = df[cols2]

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')

In [10]:
for l in list:
    df2 = df[df[' ID'] == l]
    df2 = df2.dropna(subset=['Flow(m³/s)'])
    try:
        df3 = pd.read_csv(fr'C:\Users\leachj\PycharmProjects\tinihydat\HYDAT\No RAMP\{l}_Daily_Flow_ts.csv', encoding='ANSI')
        df3['Date'] = pd.to_datetime(df3['Date'])
        df3['Date'] = df3['Date'].dt.strftime('%Y/%m/%d')
        df3 = df3.drop([len(df3) - 1, len(df3) - 2])
        df3 = df3.dropna(subset=['Flow(m³/s)'])
    except FileNotFoundError:
        df3 = pd.DataFrame(columns=cols2)
    df4 = df3.append(df2)
    df4 = df4.sort_values(by='Date')
    
    if len(df2) + len(df3) != len(df4):
        print('ERROR')
        break
    
    df4 = df4.set_index('Date')
    df5 = pd.DataFrame({'Date': pd.date_range(start=df4.index[0], end=df4.index[-1])})
    df5 = df5.set_index('Date')  
    idx = df5.index.isin(df4.index)
    df5[' ID'] = l
    df5['PARAM'] = 1
    df5.loc[idx, 'SYM'] = df4['SYM'].values
    df5.loc[idx, 'Flow(m³/s)'] = df4['Flow(m³/s)'].values
    df5 = df5.reset_index()
    df5 = df5[cols2]
    df5['Date'] = df5['Date'].dt.strftime('%Y/%m/%d')
    df5.to_csv(f'{l}_Daily_Flow_ts.csv', index=False, encoding='ANSI')
    with open(f'{l}_Daily_Flow_ts.csv', 'a') as f:
        f.write('\n')
        f.write('DISCLAIMER\n')
        f.write('"NOTICE: This application and its data are provided AS-IS.  In no event shall Environment Canada be liable for any damages whatsoever (including, without limitation, damages for loss of business profits, business interruption, loss of business information, or other pecuniary loss) arising out of the use of, or inability to use this Environment Canada product, even if Environment Canada has been advised of the possibility of such damages."\n')
    

In [11]:
df4

,ID,PARAM,Flow(m³/s),SYM
Date,,,,
2012/05/20,07CE008,1.0,6.82,NaN
2012/05/21,07CE008,1.0,6.52,NaN
2012/05/22,07CE008,1.0,5.91,NaN
2012/05/23,07CE008,1.0,5.45,NaN
2012/05/24,07CE008,1.0,5.02,NaN
...,...,...,...,...
2020/12/27,07CE008,1.0,1.78,B
2020/12/28,07CE008,1.0,1.76,B
2020/12/29,07CE008,1.0,1.75,B
